In [1]:
import os
import cv2

import numpy as np

In [2]:
def cv_show(img):
    # 图像读取
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## 获取数据

In [3]:
img1 = cv2.imread('data/imageStich/left.png')
img2 = cv2.imread('data/imageStich/right.png')

cv_show(img1)
cv_show(img2)

## Brute-Force蛮力匹配

In [4]:
sift = cv2.xfeatures2d.SIFT_create()

kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

print(des1.shape)
print(des2.shape)

(749, 128)
(898, 128)


In [5]:
# crossCheck表示两个特征点要相互匹配，A中i，B中j，要相互离对方最近
bf = cv2.BFMatcher(crossCheck=True)

matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)

In [6]:
# 画出匹配的特征点
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:30], None, flags=2)

cv_show(img3)

## K对最佳匹配

In [7]:
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

# 过滤关键点的方法，需要自己根据实际数据配置
# 这里是希望 (m/n) < 0.35，去除离群点
good = []
for m, n in matches:
    if m.distance < 0.35*n.distance:
        good.append([m])
        
img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=2)
cv_show(img3)

## 全景拼接

In [8]:
def detectAndDescribe(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 建立SIFT生成器，检测SIFT特征点并转化为Numpy数组
    descriptor = cv2.xfeatures2d.SIFT_create()
    (kps, features) = descriptor.detectAndCompute(image, None)
    kps = np.float32([kp.pt for kp in kps])
    
    return (kps, features)

In [9]:
def matchKeypoints(kpsA, kpsB, featureA, featureB, ratio, reprojThresh):
    matcher = cv2.BFMatcher()
    rawMatches = matcher.knnMatch(featureA, featureB, 2)
    
    matches = []
    for m in rawMatches:
        # 当最近距离和次距离的比值小于ratio值时，保留匹配对，并记录索引值
        if len(m) == 2 and m[0].distance < ratio * m[1].distance:
            matches.append((m[0].trainIdx, m[0].queryIdx))
    
    # 当筛选后的匹配大于4时，计算视角变换矩阵
    if len(matches) > 4:
        ptsA = np.float32([kpsA[i] for (_, i) in matches])
        ptsB = np.float32([kpsB[i] for (i, _) in matches])
        
        # 计算视角变换矩阵
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)
        
        return (matches, H, status)
    
    return None

In [10]:
def drawMatches(imageA, imageB, kpsA, kpsB, matches, status):
    # 初始化可视化图片，将A、B图左右连接到一起
    (hA, wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype='uint8')
    vis[0: hA, 0: wA] = imageA
    vis[0: hB, wA:] = imageB
    
    # 联合遍历，画出匹配对
    for ((trainIdx, queryIdx), s) in zip(matches, status):
        if s == 1:
            ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
            ptB = (int(kpsB[trainIdx][0])+wA, int(kpsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)
    
    return vis

In [11]:
def stitch(images, ratio=0.75, reprojThresh=4.0, showMatches=False):
    (imageB, imageA) = images
    (kpsA, featuresA) = detectAndDescribe(imageA)
    (kpsB, featuresB) = detectAndDescribe(imageB)
    
    M = matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh)
    if M is None:
        return None
    
    # 提取匹配结果，并对A进行视角变换
    (matches, H, status) = M
    result = cv2.warpPerspective(imageA, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
    cv_show(result)
    
    # 将图片B传入result图片最左端
    result[0: imageB.shape[0], 0: imageB.shape[1]] = imageB
    cv_show(result)
    
    if showMatches:
        vis = drawMatches(imageA, imageB, kpsA, kpsB, matches, status)
        return (result, vis)
    
    return result

In [12]:
imgA = cv2.imread('data/imageStich/left.png')
imgB = cv2.imread('data/imageStich/right.png')

(result, vis) = stitch([imgA, imgB], showMatches=True)